<h1>Regressie (jaar)</h1>
begin met basics van maand regressie

In [1]:
import pandas as pd
import pyodbc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Maak verbinding met de Microsoft Access-database
conn_str_sales = (
    r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=C:\\Users\\Burto\\Downloads\\GO-databases DEDS-week 4 & 5\\HC code\Week 10\\GO-databases schoon\\go_sales_schoon.accdb;"
    r"ExtendedAnsiSQL=1;"
)
conn_SALES = pyodbc.connect(conn_str_sales)

# Haal gegevens op uit de database
df_returnedItems = pd.read_sql_query("SELECT * FROM returned_item", conn_SALES)
df_details = pd.read_sql_query("SELECT * FROM order_details", conn_SALES)
df_header = pd.read_sql_query('SELECT * FROM order_header', conn_SALES)

#JOIN tabel
df = pd.merge(df_returnedItems, df_details, on='ORDER_DETAIL_CODE')
df = pd.merge(df,df_header, on='ORDER_NUMBER')

#Selecteer alleen de relevante informatie
df = df.dropna()
df = df[['ORDER_DATE','RETURN_QUANTITY']]

C:\Users\Burto\AppData\Local\Temp\ipykernel_19748\60034587.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_returnedItems = pd.read_sql_query("SELECT * FROM returned_item", conn_SALES)
C:\Users\Burto\AppData\Local\Temp\ipykernel_19748\60034587.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_details = pd.read_sql_query("SELECT * FROM order_details", conn_SALES)
C:\Users\Burto\AppData\Local\Temp\ipykernel_19748\60034587.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_header = pd.read_sql_query('SELECT *

<h2>Voeg maanden toe</h2>

In [2]:
# Zet ORDER_DATE om naar maanden
df['MONTH'] = pd.to_datetime(df['ORDER_DATE']).dt.month

# Groepeer gegevens per maand
df_grouped = df.groupby(['MONTH']).sum().reset_index()

C:\Users\Burto\AppData\Local\Temp\ipykernel_19748\3333002408.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped = df.groupby(['MONTH']).sum().reset_index()


<h2>Regressiemodel</h2>
<p>We kunnen beginnen met het toevoegen van een nieuwe kolom met het jaartal van de bestelling en het verwijderen van de ORDER_DATE-kolom</p>

In [3]:
# Gebruik LINEAR REGRESSION om de relatie tussen maand en geretourneerde hoeveelheid te bepalen
X = df_grouped['MONTH'].values.reshape(-1, 1)
y = df_grouped['RETURN_QUANTITY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)

# Voorspel het aantal geretourneerde producten voor de maand januari
predicted_returns = model.predict([[1]])
print(predicted_returns)

[[-1.2370476e+174]]


C:\Users\Burto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\linalg\_basic.py:1208: RuntimeWarning: overflow encountered in square
  resids = np.sum(np.abs(x[n:])**2, axis=0)


<h3>Bereken R2 score</h3>
<p>Een hogere score is beter.<br/>Laten we de code testen door 2024 in te vullen, kijken wat er uit komt.</p>

In [4]:
# Bereken de R2-score
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# Print de de R2-score
print(r2)

nan


C:\Users\Burto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:927: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\Burto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:929: RuntimeWarning: overflow encountered in square
  weight * (y_true - np.average(y_true, axis=0, weights=sample_weight)) ** 2
C:\Users\Burto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:633: RuntimeWarning: invalid value encountered in divide
  numerator[valid_score] / denominator[valid_score]


<h2>Polynomial poging</h2>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

model_poly = LinearRegression()
model_poly.fit(X_poly, y)
